In [32]:
import numpy as np
import control as ctrl
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, interact_manual, Output

# Importações necessárias e classe ControlFunctions
from functions import ControlFunctions

# Área de output para os gráficos
output_plot = Output()

def setup_system():
    fx = ControlFunctions()

    g = 9.8
    M_vazio = 29000
    L = 2.9
    C = 17
    H = 2
    rho_fluido = 715
    hf = 2 * 0.6
    mf = rho_fluido * L * C * hf

    m0 = mf - fx.massa_i(mf, 1, hf, L) - fx.massa_i(mf, 2, hf, L)
    m1 = fx.massa_i(mf, 1, hf, L)
    m2 = fx.massa_i(mf, 2, hf, L)
    m3 = fx.massa_i(mf, 3, hf, L)

    k1 = fx.calcular_ki(hf, L, g, mf, 1)
    k2 = fx.calcular_ki(hf, L, g, mf, 2)
    h0 = fx.calcular_hi(L, hf, 0)
    h1 = fx.calcular_hi(L, hf, 1)
    h2 = fx.calcular_hi(L, hf, 2)
    h3 = fx.calcular_hi(L, hf, 3)

    omega1 = np.sqrt(k1 / m1)
    omega2 = np.sqrt(k2 / m2)
    zeta = ((6 * 10**(-6)) / (np.sqrt(g) * L**(3/2)))**(1/2)
    c1 = fx.calcular_ci(m1, omega1, zeta, 1)
    c2 = fx.calcular_ci(m2, omega2, zeta, 1)

    I0 = fx.momento_inercia(M_vazio, L, H)
    If = I0 + -m0 * h0**2 + m1 * h1**2 + m2 * h2**2

    A = np.array([[0, 0, 0, 1, 0, 0],
                  [0, 0, 0, 0, 1, 0],
                  [0, 0, 0, 0, 0, 1],
                  [-k1 / m1, 0, g, -c1 / m1, 0, 0],
                  [0, -k2 / m2, g, 0, -c2 / m2, 0],
                  [g * m1 / If, g * m2 / If, -15000 * L**2 / (2 * If), 0, 0, -120 * L**2 / (2 * If)]])
    B = np.array([[0], [0], [0], [0], [0], [1 / If]])
    C = np.array([[0, 0, 1, 0, 0, 0]])
    D = np.array([[0]])

    return A, B, C, D

def plot_response(real1, imag1, real2, imag2, real3, imag3):
    A, B, C, D = setup_system()
    poles = [
        complex(real1, imag1), complex(real1, -imag1),
        complex(real2, imag2), complex(real2, -imag2),
        complex(real3, imag3), complex(real3, -imag3)
    ]
    K = ctrl.place(A, B, poles)
    A_new = A - B.dot(K)
    sys = ctrl.ss(A_new, B, C, D)

    with output_plot:
        output_plot.clear_output(wait=True)
        plt.figure(figsize=(12, 6))
        ctrl.root_locus(sys, plot=True)
        plt.title('Root Locus for Poles: {}'.format(poles))
        plt.xlabel('Real Axis')
        plt.ylabel('Imaginary Axis')
        plt.grid(True)
        plt.show()

        T, Y = ctrl.step_response(sys)
        plt.figure(figsize=(12, 6))
        plt.plot(T, Y)
        plt.title('Step Response')
        plt.xlabel('Time (seconds)')
        plt.ylabel('Output')
        plt.grid(True)
        plt.show()

interact_manual = interact.options(manual=True, manual_name="Generate Plots")
interact_manual(plot_response,
                real1=FloatSlider(value=-10, min=-50, max=-1, step=1, description='Real 1'),
                imag1=FloatSlider(value=10, min=0, max=50, step=1, description='Img 1'),
                real2=FloatSlider(value=-20, min=-50, max=-1, step=1, description='Real 2'),
                imag2=FloatSlider(value=20, min=0, max=50, step=1, description='Img 2'),
                real3=FloatSlider(value=-30, min=-50, max=-1, step=1, description='Real 3'),
                imag3=FloatSlider(value=30, min=0, max=50, step=1, description='Img 3'))

display(output_plot)


interactive(children=(FloatSlider(value=-10.0, description='Real 1', max=-1.0, min=-50.0, step=1.0), FloatSlid…

Output()